In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Load data =(Untuk memuat dataset)
data = pd.read_csv('VCT_2024.csv') #data yang dimuat berasal dari CSV file menggunakan pandas

# Drop irrelevant columns =(Menghapus kolom yang tidak digunakan untuk modeling)
drop_cols = ['Region', 'Player', 'Team Abbreviated', 'Event', 'CL', 'R'] #Merupakan data yang tidak digunakan atau di drop
data = data.drop(columns=drop_cols)

# Define numeric columns (including ACS) =(Daftar seluruh fitur numerik yang akan digunakan didalam model, termasuk ACS yang utama)
numeric_cols = ['ACS', 'K:D', 'KAST', 'ADR', 'KPR', 'APR', 'FKPR', 'FDPR', 'HS%', 'CL%', 'CW', 'CP']
#=(Merupakan kolom numerik. 
# Average Combat Score (ACS)=Metrik kunci performa, Kill to Death Ratio(K:D)=Efisiensi Pemain, Kill Assist Survive, Trade %(KAST)=Metrik Konsistensi, Average Damage per Round(ADR)=Hasil rata-rata kerusakan yang didapatkan tiap ronde, Kill/Assists per Round(KPR/APR)=Metrik agresi gaya permainan pemain, First Kills/Death per Round(FKPR/FKDR)=Entry Fragging Impact, mengacu pada pengaruh yang diberikan pemain ketika mereka dapat/gagal melakukan first kill saat tim sedang execute ke lokasi, Headshot Percentage(HS%)=Presentasi kepresisian dalam membidik, Clutch Win Percentage(CL%)=presentasi keberhasilan memenangkan clutch, Clutch Won(CW)=Jumlah situasi clutch yg berhasil dimenangkan, Clutch Played(CP)=jumlah total situasi clutch yang dimainkan. (Clutch adalah istilah ketika pemain dalam kondisi tidak menguntungkan atau berada dalam tekanan tinggi tapi mampu mengalahkan semua lawan dan menyelamatkan ronde untuk timnya)

# --- Define Binary Target (Win=1, Loss=0) --- =(mendefinisikan masalah klasifikasi biner (Menang/Kalah). 
# Win =1 jika ACS pemain lebih dari median ACS atau diatas 50% performa dan Win = 0 jika ACS kurang dari median atau dibawah 50%. Seperti ini karena kumpulan data mungkin tidak memiliki hasik pertandingan yang sebenarnya sehingga hal ini secara artifisial menciptakan tugas klasifikasi berdasarkan performa pemain)
data['Win'] = (data['ACS'] >= data['ACS'].median()).astype(int)
y = data['Win']
X = data.drop(columns=['Win'])

In [ ]:
# --- Preprocessing --- (Menyiapkan data untuk machine learning dengan menangani data kategoris, nilai yang hilang, dan penskalaan fitur)
# 1. Encode categorical variables (Team)
X = pd.get_dummies(X, columns=['Team'], drop_first=True )
#kolom team dikodekan one-hot menggunakan pd.get_dummies
#drop_first true menghindari jebakan variabel dummy (menghapus satu kolom untuk mencegah multikolinearitas)

# 2. Handle missing values using SimpleImputer (Mengisi nilai yang hilang (NaN) untuk mencegah kesalahan model)
imputer = SimpleImputer(strategy='mean')  # Replace NaNs with mean
X[numeric_cols] = imputer.fit_transform(X[numeric_cols])
#Pakai mean untuk asumsi nilai yang hilang secara random (MAR) dan dapat didekati dengan rata-rata

# 3. Normalize numeric features (Feature Scalling MinMaxScaler) =(Mernormalkan semua fitur ke [0,1] range)
scaler = MinMaxScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
#Penskalaan begini karena SVM menggunakan kalkulasi berbasis jarak, jadi fitur harus berada pada skala yang sama
#alternatif: StandardScaler (normalizes to mean=0, std=1).

# 4. Final check for NaNs (should print 0) =(Memastikan tidak ada nilai yang hilang, jika preprocesing nya benar maka nilainya 0)
print("Remaining NaNs:", X.isna().sum().sum())

Remaining NaNs: 0


In [5]:
# print("NaN counts per column:")
# print(X.isna().sum())

In [ ]:
# Train-test split (Membagi data untuk training 70% dan testing 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#test_size=0.3: 30% untuk pengujian, 70% untuk pelatihan.
#random_state=42: memastikan reproduktifitas (pemisahan yang sama setiap kali dijalankan).
#Pemisahan dilakukan untuk mengevaluasi kinerja model pada data yang tidak terlihat.

# --- Train SVM ---
# Using a pipeline to ensure no data leakage (Creating the pipeline)=(menggabungkan preprocessing dan modeling ke satu pipeline)
svm_pipeline = make_pipeline(
    SVC(kernel='rbf', C=1.0, probability=True)
)
#SVC = Core model
#kernel="rbf" = memungkinkan batasan non linear decision
#C=1.0 = Parameter regulasi (Semakin tinggi = semakin sedikit regulasi)
#probability=true = memungkinkan estimasi probabilitas (predict_proba)
svm_pipeline.fit(X_train, y_train) #menyesuaikan SVM dengan data pelatihan
#Model mempelajari hyperplane optimal yang memisahkan "win" vs "loss"
#Menggunakan kernel untuk menangani hubungan non-linear

# Evaluate
y_pred = svm_pipeline.predict(X_test)
#Mengeneralisasi prediksi pada test set
print("Accuracy:", accuracy_score(y_test, y_pred))
#Mengukur seberapa benar prediksi
print(classification_report(y_test, y_pred))
#Memberikan detail metrik performa
#presisi =berapa banyak prediksi kemenangan yang benar?
#recall= berapa banyak kemenangan yang benar-benar terdeteksi?
#F1-Score= Keseimbangan/rata-rata metrik
#Support= Jumlah dari sample di tiap class

Accuracy: 0.8154506437768241
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       117
           1       0.87      0.74      0.80       116

    accuracy                           0.82       233
   macro avg       0.82      0.82      0.81       233
weighted avg       0.82      0.82      0.81       233

